这个是用于生成negative example和positive example

用于训练判别模型

In [10]:
from tqdm import tqdm
import random

In [2]:
%cd ../../

/root/code/bioner/KnowledgeEmbedding


In [7]:
from src.utils.dataset_utils import KGProcessor

processor = KGProcessor()
data_dir = "./dataset/drugbank"


train_triples = processor.get_train_triples("./dataset/drugbank")
dev_triples = processor.get_dev_triples("./dataset/drugbank")

读取:./dataset/drugbank/train.tsv文件: 17125it [00:00, 541472.85it/s]
读取:./dataset/drugbank/dev.tsv文件: 1903it [00:00, 591679.80it/s]


In [12]:
entity_list = processor.get_entities(data_dir)
all_triples = train_triples + dev_triples
all_triples_str_set = set()
drug_entites_set = set()
target_entites_set = set()
for triple in all_triples:
    triple_str = '\t'.join(triple)
    drug_entites_set.add(triple[0])
    target_entites_set.add(triple[-1])
    all_triples_str_set.add(triple_str)

In [13]:
drug_entites_li = list(drug_entites_set)
target_entities_li = list(target_entites_set)

In [14]:
train_positive_examples = []
train_negative_examples = []
for triple in tqdm(train_triples, total=len(train_triples)):
    head = triple[0].lower()
    relation = triple[1].lower()
    tail = triple[2].lower()
    # print(test_triple, head, relation, tail)
    # 给这里面加一个正确的triple
    train_positive_examples.append([head,relation,tail,1])
    if random.random()>0.5:
        corrupt_ent = random.choice(drug_entites_li)
        tmp_triple = [corrupt_ent, relation, tail]
        tmp_triple_str = '\t'.join(tmp_triple)
        if tmp_triple_str not in all_triples_str_set:
            # may be slow
            train_negative_examples.append([corrupt_ent, relation, tail,0])
    else:
        corrupt_ent = random.choice(target_entities_li)
        tmp_triple = [head, relation, corrupt_ent]
        tmp_triple_str = '\t'.join(tmp_triple)
        if tmp_triple_str not in all_triples_str_set:
            # may be slow
            train_negative_examples.append([head, relation, corrupt_ent,0])


100%|██████████| 17125/17125 [00:00<00:00, 83985.34it/s]


In [15]:
dev_positive_examples = []
dev_negative_examples = []
for triple in tqdm(dev_triples, total=len(dev_triples)):
    head = triple[0].lower()
    relation = triple[1].lower()
    tail = triple[2].lower()
    # print(test_triple, head, relation, tail)
    # 给这里面加一个正确的triple
    dev_positive_examples.append([head,relation,tail,1])
    if random.random()>0.5:
        corrupt_ent = random.choice(drug_entites_li)
        tmp_triple = [corrupt_ent, relation, tail]
        tmp_triple_str = '\t'.join(tmp_triple)
        if tmp_triple_str not in all_triples_str_set:
            # may be slow
            dev_negative_examples.append([corrupt_ent, relation, tail,0])
    else:
        corrupt_ent = random.choice(target_entities_li)
        tmp_triple = [head, relation, corrupt_ent]
        tmp_triple_str = '\t'.join(tmp_triple)
        if tmp_triple_str not in all_triples_str_set:
            # may be slow
            dev_negative_examples.append([head, relation, corrupt_ent,0])


100%|██████████| 1903/1903 [00:00<00:00, 126108.11it/s]


In [18]:
dev_negative_examples[0]

['GSK-364735', 'target', 'c-x-c chemokine receptor type 4', 0]

In [19]:
train_examples = []
dev_examples = []
train_examples.extend(train_positive_examples)
train_examples.extend(train_negative_examples)

dev_examples.extend(dev_positive_examples)
dev_examples.extend(dev_negative_examples)

In [21]:
import pickle
with open("train_examples.pk",'wb') as f:
    pickle.dump(train_examples,f)

In [22]:
import pickle
with open("dev_examples.pk",'wb') as f:
    pickle.dump(dev_examples,f)

In [23]:
!pwd

/root/code/bioner/KnowledgeEmbedding
